This notebook parses some of the CARDiPS files to take only data that we need for this project.

In [6]:
import glob
import os
import subprocess

import cdpybio as cpb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import ciepy as cpy
import projectpy as ppy

%matplotlib inline

In [34]:
array_meta = pd.read_csv(os.path.join(cpy.root, 'data', 'array_metadata.csv'), index_col=-1)
wgs_meta = pd.read_table(os.path.join(cpy.root, 'data', 'wgs_metadata.tsv'), index_col=0, 
                         squeeze=True)
wgs_meta.index.name = 'wgs_id'
fn = ('/raid3/projects/CARDIPS/pipeline/RNAseq/150512_150522_150527_metadata.tsv')
rna_meta = pd.read_table(fn, index_col=0)

# Metadata

## Subject

In [41]:
subject_metadata = pd.read_csv(os.path.join(cpy.root, 'data', 'subject_metadata.csv'),
                               index_col=-1)
conv = pd.Series(subject_metadata.index, index=subject_metadata.Name)
for c in ['Father', 'Mother', 'Twin']:
    for i in subject_metadata[c].dropna().index:
        subject_metadata.ix[i, c] = conv[subject_metadata.ix[i, c]]

## RNA

In [32]:
pd.crosstab(rna_meta.study_ipsc_eqtl, rna_meta.censor)

censor,False,True
study_ipsc_eqtl,,
0,44,2
1,193,31


In [10]:
rna_meta = rna_meta[rna_meta.censor == False]
rna_meta = rna_meta[rna_meta.study_ipsc_eqtl == 1]
rna_meta = rna_meta.reset_index().merge(wgs_meta.reset_index(), on='subject').set_index('index')
rna_meta = rna_meta.dropna(subset=['wgs_id'])
rna_meta.to_csv(os.path.join(cpy.root, 'data', 'rna_seq_metadata.tsv'), sep='\t')

I'm currently missing WGS data for 11 RNA-seq samples. There are 31 samples that are censored.

# Gene Counts

In [11]:
fns = glob.glob(os.path.join('/raid3/projects/CARDIPS/pipeline/RNAseq/*XX/cdeboever/'
                             'results/*_counts/gene_counts.tsv'))
count_fns = []
for fn in fns:
    s = fn.split(os.path.sep)[-2].split('_')[0]
    if s in rna_meta.index:
        count_fns.append(fn)

In [12]:
counts = [pd.read_table(fn, index_col=0, header=None, squeeze=True) for fn in count_fns]
sample_names = [fn.split(os.path.sep)[-2].split('_')[0] for fn in count_fns]
d = {key: value for (key, value) in zip(sample_names, counts)}
counts = pd.DataFrame(d)
counts.index.name = 'gene'
counts.to_csv(os.path.join(cpy.root, 'data', 'gene_counts.tsv'), sep='\t')